In [ ]:
import collections
import os

# Limpiar consola según sistema operativo
os.system('cls' if os.name == 'nt' else 'clear')

# ===== COLORES Y SIMBOLOS =====
RESET = "\033[0m"
ROJO = "\033[31m"    # Obstáculos (X)
VERDE = "\033[32m"   # Inicio/Fin (S/E)
AZUL = "\033[34m"    # Agua (~)
AMARILLO = "\033[33m" # Ruta (*)
GRIS = "\033[90m"    # Libre (.)

# Mapeo de valores según el README
LIBRE = 0
EDIFICIO = 1
AGUA = 2
BLOQUEADO = 3

SIMBOLOS = {
    LIBRE: ".",
    EDIFICIO: "X",
    AGUA: "~",
    BLOQUEADO: "B",
    "RUTA": "*"
}

# ===== LÓGICA DE BÚSQUEDA (BFS) =====
def buscar_ruta(mapa, inicio, fin):
    filas = len(mapa)
    cols = len(mapa[0])
    cola = collections.deque([inicio])
    visitados = {inicio: None}  # Guarda el nodo padre para reconstruir la ruta

    while cola:
        actual = cola.popleft()

        if actual == fin:
            # Reconstruir camino
            ruta = []
            while actual:
                ruta.append(actual)
                actual = visitados[actual]
            return ruta[::-1]

        x, y = actual
        # Movimientos: Arriba, Abajo, Izquierda, Derecha
        for dx, dy in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
            nx, ny = x + dx, y + dy

            if 0 <= nx < filas and 0 <= ny < cols:
                # Solo transitable si es LIBRE (0) o el DESTINO
                # (Se pueden incluir otros pero el README pide esquivar obstáculos)
                if (nx, ny) not in visitados and mapa[nx][ny] == LIBRE:
                    visitados[(nx, ny)] = actual
                    cola.append((nx, ny))
                elif (nx, ny) == fin: # Permitir llegar al destino aunque tenga otro valor
                    visitados[(nx, ny)] = actual
                    cola.append((nx, ny))
                    
    return None

# ===== VISUALIZACIÓN =====
def imprimir_mapa(mapa, ruta=[], inicio=None, fin=None):
    os.system('cls' if os.name == 'nt' else 'clear')
    print(f"{AMARILLO}--- THE HUDDLE: CALCULADORA DE RUTAS ---{RESET}\n")
    
    for r in range(len(mapa)):
        for c in range(len(mapa[0])):
            pos = (r, c)
            char = SIMBOLOS[mapa[r][c]]
            color = GRIS

            if pos == inicio:
                print(f"{VERDE}S{RESET}", end=" ")
            elif pos == fin:
                print(f"{VERDE}E{RESET}", end=" ")
            elif ruta and pos in ruta:
                print(f"{AMARILLO}*{RESET}", end=" ")
            elif mapa[r][c] == EDIFICIO or mapa[r][c] == BLOQUEADO:
                print(f"{ROJO}X{RESET}", end=" ")
            elif mapa[r][c] == AGUA:
                print(f"{AZUL}~{RESET}", end=" ")
            else:
                print(f"{color}{char}{RESET}", end=" ")
        print()

# ===== FLUJO PRINCIPAL =====
def main():
    try:
        f = int(input("Define alto del mapa: "))
        c = int(input("Define ancho del mapa: "))
        mapa = [[LIBRE for _ in range(c)] for _ in range(f)]

        print("\nConfigura coordenadas (fila columna):")
        start_f, start_c = map(int, input("Inicio: ").split())
        end_f, end_c = map(int, input("Destino: ").split())
        
        inicio = (start_f, start_c)
        fin = (end_f, end_c)

        while True:
            ruta_encontrada = buscar_ruta(mapa, inicio, fin)
            imprimir_mapa(mapa, ruta_encontrada, inicio, fin)

            if not ruta_encontrada:
                print(f"\n{ROJO}¡ALERTA! No hay ruta posible.{RESET}")

            print("\nOpciones: [1] Agregar Obstáculo | [2] Agregar Agua | [3] Salir")
            opc = input("Selecciona: ")

            if opc == "3": break
            
            try:
                obs_f, obs_c = map(int, input("Coordenada (fila columna): ").split())
                if (obs_f, obs_c) in [inicio, fin]:
                    input("No puedes bloquear el inicio o fin. Presiona Enter...")
                    continue
                
                tipo = EDIFICIO if opc == "1" else AGUA
                mapa[obs_f][obs_c] = tipo
            except:
                input("Coordenada inválida. Presiona Enter...")

    except Exception as e:
        print(f"Error de configuración: {e}")

if __name__ == "__main__":
    main()